### TASK 1: Add a Launch Site Drop-down Input Component

We have four different launch sites and we would like to first see which one has the largest success count. Then,
we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented dcc.Dropdown(id='site-dropdown',...) input with following attributes:

1. id attribute with value site-dropdown

2. options attribute is a list of dict-like option objects (with label and value attributes). You can set the label and value all to be the launch site names in the spacex_df and you need to include the default All option. e.g.,   options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]

3. value attribute with default dropdown value to be ALL meaning all sites are selected

4. placeholder attribute to show a text description about this input area, such as Select a Launch Site here

5. searchable attribute to be True so we can enter keywords to search launch sites

### TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

The general idea of this callback function is to get the selected launch site from site-dropdown and render
a pie chart visualizing launch success counts

Let’s add a callback function in spacex_dash_app.py including the following application logic:

1. Input is set to be the site-dropdown dropdown, i.e., Input(component_id='site-dropdown', component_property='value')
2. Output to be the graph with id success-pie-chart, i.e., Output(component_id='success-pie-chart', component_property='figure')
3. A If-Else statement to check if ALL sites were selected or just a specific launch site was selected.
    
    3.1. If ALL sites are selected, we will use all rows in the dataframe spacex_df to render and return a pie chart graph to show the total success launches (i.e., the total count of class column).
    
    3.2. If a specific launch site is selected, you need to filter the dataframe spacex_df first in order to include the only data for the selected site. Then, render and return a pie chart graph to show the success (class=1) count and failed (class=0) count for the selected site.

### TASK 3: Add a Range Slider to Select Payload

Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we want to be able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented dcc.RangeSlider(id='payload-slider',...) input with the following attribute:

1. id to be payload-slider

2. min indicating the slider starting point, we set its value to be 0 (Kg)

3. max indicating the slider ending point to, we set its value to be 10000 (Kg)

4. step indicating the slider interval on the slider, we set its value to be 1000 (Kg)

5. value indicating the current selected range, we could set it to be min_payload and max_payload

### TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., class column).
As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

In addition, we want to color-label the Booster version on each scatter point so that we may observe mission outcomes with different boosters.

Now, let’s add a call function including the following application logic:

1. Input to be [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]. Note that we have two input components, one to receive selected launch site and another to receive selected payload range

2. Output to be Output(component_id='success-payload-scatter-chart', component_property='figure')

3. A If-Else statement to check if ALL sites were selected or just a specific launch site was selected

    3.1. If ALL sites are selected, render a scatter plot to display all values for variable Payload Mass (kg) and variable class. In addition, the point color needs to be set to the booster version i.e., color="Booster Version Category"

    3.2. If a specific launch site is selected, you need to filter the spacex_df first, and render a scatter chart to show values Payload Mass (kg) and class for the selected site, and color-label the point using Boosster Version Category likewise.

In [29]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
spacex_df['Launch Site'].unique()

launch_sites = spacex_df['Launch Site'].unique()
launch_sites = list(launch_sites)
launch_sites.sort()
launch_sites.insert(0, 'ALL')

# Add a new column 'Outcome' that maps 1 to 'Success' and 0 to 'Failure'
spacex_df['Outcome'] = spacex_df['class'].map({1:'Success', 0:'Failure'})

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category,Outcome
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0,Failure
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0,Failure
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0,Failure
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0,Failure
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0,Failure


In [30]:
# Create a dash application
app = dash.Dash(__name__)

app.layout = html.Div([
    # TASK 1: Add a title, and a dropdown list
    # The title of our dashboard
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                    'font-size': 40}),
    # The dropdown list
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),
    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("The total number of successful launches is: ", id='success-pie-chart-total'),
    html.Br(),
    # TASK 3: Add a slider to select payload range
    html.P("Select a payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]
    ),
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class', title='Total Success Launches for Site ' + selected_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if selected_site == 'ALL':
        fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.scatter(filtered_df[mask], x='Payload Mass (kg)', y='class', color='Outcome',
                         title='Correlation between Payload and Success for Site ' + selected_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8050)
    

### Finding Insights Visually

Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

In [31]:
# 1. Which site has the largest successful launches?
# code to find the site with the largest successful launches
site1 = spacex_df.groupby(['Launch Site']).sum().sort_values(by='class', ascending=False).index[0]
site1

'KSC LC-39A'

In [25]:
# 2. Which site has the highest launch success rate?
# code to find the answer
site2 = spacex_df.groupby('Launch Site')['class'].mean().sort_values(ascending=False).index[0]
site2

'KSC LC-39A'

In [37]:
# 3. Which payload range(s) has the highest launch success rate?
# 4. Which payload range(s) has the lowest launch success rate?

# find the payload range with the highest/lowest success rate
payload_df = spacex_df.groupby('Payload Mass (kg)')['class'].mean().reset_index()
payload_df = payload_df.sort_values(by='class', ascending=False)
# payload_df


In [38]:
# 5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
# code to find the answer
booster_df = spacex_df.groupby('Booster Version')['class'].mean().reset_index()
booster_df = booster_df.sort_values(by='class', ascending=False)
# booster_df

# FT has the highest launch success rate